In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
limits = pd.read_excel("pipeline_limits.xlsx")
routes = pd.read_excel("pipeline_routes.xlsx")

In [3]:
limits["from_to"] = limits["From Code"] + "_" + limits["To Code"]
limits.head(2)

,From Country,From Code,To Country,To Code,GWh/d,from_to
0,Algeria,DZ,Italy,IT,448.0,DZ_IT
1,Algeria,DZ,Spain,ES,245.0,DZ_ES


In [4]:
routes.head(2)

,Country,Code,Route
0,Austria,AT,AT->DE
1,Algeria,DZ,DZ->TN->IT->CH->DE


In [5]:
not_in_limits = [x for x in routes["Country"].unique() if x not in limits["From Country"].unique() and x not in limits["To Country"]]
not_in_routes = [x for x in limits["From Country"].unique() if x not in routes["Country"].unique()]
print("Not in Limits:", not_in_limits)
print("Not in Routes:", not_in_routes)

Not in Limits: ['Azerbaijan', 'Iraq', 'Iran', 'Libya', 'Morocco', 'Moldova', 'Russian Federation', 'Syria', 'Turkmenistan', 'Tunisia', 'Turkey']
Not in Routes: ['Bosnia-Herznegovina', 'Germany', 'Latvia', 'Luxembourg', 'Serbia']


In [6]:
routes["First Transport"] = routes["Route"].apply(lambda x: x[4:6])
#routes_capped = routes[~routes["Country"].isin(not_in_limits)].copy()

routes_capped["Downstream"] = routes_capped["Code"].apply(lambda country: routes_capped.loc[routes_capped["Route"].apply(lambda x: "->"+country in x), "Code"].to_list())
routes_capped["First Transport"] = routes_capped["Route"].apply(lambda x: x[4:6])
routes_capped["from_to"] = routes_capped["Code"] + "_" + routes_capped["First Transport"]

In [7]:
routes["Downstream"] = routes["Code"].apply(lambda country: routes.loc[routes["Route"].apply(lambda x: "->"+country in x), "Code"].to_list())
routes["First Transport"] = routes["Route"].apply(lambda x: x[4:6])
routes["from_to"] = routes["Code"] + "_" + routes["First Transport"]

In [8]:
routes

,Country,Code,Route,First Transport,Downstream,from_to
0,Austria,AT,AT->DE,DE,"[AZ, BG, GR, HU, IQ, IR, MD, RO, SK, SY, TM, TR]",AT_DE
1,Algeria,DZ,DZ->TN->IT->CH->DE,TN,[],DZ_TN
2,Azerbaijan,AZ,AZ->TR->GR->BG->RO->HU->SK->AT->DE,TR,[],AZ_TR
3,Estonia,EE,EE->LT->PL->DE,LT,[],EE_LT
4,Belgium,BE,BE->DE,DE,[],BE_DE
5,Bulgaria,BG,BG->RO->HU->SK->AT->DE,RO,"[AZ, GR, IQ, IR, SY, TM, TR]",BG_RO
6,Czechia,CZ,CZ->DE,DE,[],CZ_DE
7,Denmark,DK,DK->DE,DE,[],DK_DE
8,Spain,ES,ES->FR->DE,FR,"[MA, PT]",ES_FR
9,Finland,FI,FI->DE,DE,[],FI_DE


In [11]:
pd.merge(routes, limits, how = "left", on = "from_to").fillna(0)

,Country,Code,Route,First Transport,Downstream,from_to,From Country,From Code,To Country,To Code,GWh/d
0,Austria,AT,AT->DE,DE,"[AZ, BG, GR, HU, IQ, IR, MD, RO, SK, SY, TM, TR]",AT_DE,Austria,AT,Germany,DE,150.00000
1,Algeria,DZ,DZ->TN->IT->CH->DE,TN,[],DZ_TN,0,0,0,0,0.00000
2,Azerbaijan,AZ,AZ->TR->GR->BG->RO->HU->SK->AT->DE,TR,[],AZ_TR,0,0,0,0,0.00000
3,Estonia,EE,EE->LT->PL->DE,LT,[],EE_LT,0,0,0,0,0.00000
4,Belgium,BE,BE->DE,DE,[],BE_DE,Belgium,BE,Germany,DE,105.50000
5,Bulgaria,BG,BG->RO->HU->SK->AT->DE,RO,"[AZ, GR, IQ, IR, SY, TM, TR]",BG_RO,Bulgaria,BG,Romania,RO,125.08642
6,Czechia,CZ,CZ->DE,DE,[],CZ_DE,Czechia,CZ,Germany,DE,231.60000
7,Denmark,DK,DK->DE,DE,[],DK_DE,Denmark,DK,Germany,DE,290.00000
8,Spain,ES,ES->FR->DE,FR,"[MA, PT]",ES_FR,Spain,ES,France,FR,216.00000
9,Finland,FI,FI->DE,DE,[],FI_DE,Finland,FI,Germany,DE,504.00000
